Using chinook.db write pandas code.

Customer Purchases Analysis:
Find the total amount spent by each customer on purchases (considering invoices).
Identify the top 5 customers with the highest total purchase amounts.
Display the customer ID, name, and the total amount spent for the top 5 customers.

In [ ]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('chinook.db')
query = """SELECT 
    c.CustomerId,
    c.FirstName || ' ' || c.LastName AS CustomerName,
    SUM(i.Total) AS TotalSpent
FROM 
    Customer c
JOIN 
    Invoice i ON c.CustomerId = i.CustomerId
GROUP BY 
    c.CustomerId
ORDER BY 
    TotalSpent DESC
LIMIT 5
"""
customers = pd.read_sql_query(query,connection)
print(customers)

    CustomerId  FirstName      LastName  \
0            1       Luís     Gonçalves   
1            2     Leonie        Köhler   
2            3   François      Tremblay   
3            4      Bjørn        Hansen   
4            5  František   Wichterlová   
5            6     Helena          Holý   
6            7     Astrid        Gruber   
7            8       Daan       Peeters   
8            9       Kara       Nielsen   
9           10    Eduardo       Martins   
10          11  Alexandre         Rocha   
11          12    Roberto       Almeida   
12          13   Fernanda         Ramos   
13          14       Mark       Philips   
14          15   Jennifer      Peterson   
15          16      Frank        Harris   
16          17       Jack         Smith   
17          18   Michelle        Brooks   
18          19        Tim         Goyer   
19          20        Dan        Miller   
20          21      Kathy         Chase   
21          22    Heather       Leacock   
22         

Album vs. Individual Track Purchases:
Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("chinook.db")

query = """
SELECT
    il.CustomerId,
    t.AlbumId,
    il.TrackId
FROM
    InvoiceLine il
JOIN
    Track t ON il.TrackId = t.TrackId
JOIN
    Invoice i ON il.InvoiceId = i.InvoiceId
"""

df = pd.read_sql_query(query, conn)

album_track_counts = pd.read_sql_query(
    "SELECT AlbumId, COUNT(*) as TotalTracks FROM Track GROUP BY AlbumId", conn)

df = df.merge(album_track_counts, on="AlbumId")

customer_album = df.groupby(['CustomerId', 'AlbumId']).agg({
    'TrackId': 'nunique',
    'TotalTracks': 'first'
}).reset_index()

customer_album['FullAlbum'] = customer_album['TrackId'] == customer_album['TotalTracks']

customer_summary = customer_album.groupby('CustomerId')['FullAlbum'].any().reset_index()

customer_summary['Preference'] = customer_summary['FullAlbum'].apply(
    lambda x: 'Full Album' if x else 'Individual Tracks')

summary = customer_summary['Preference'].value_counts(normalize=True) * 100
print(summary)